In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargo la base limpia que ya se había generado en la etapa de preprocesamiento
df = pd.read_csv("df_cleaned.csv")

# Defino el target del modelo 2 como el logaritmo natural del precio
# (mismo target conceptual del modelo 1, pero acá lo optimizamos para desempeño)
y_nn = np.log(df["price"])

# Tomo solo las columnas numéricas como variables de entrada
# (incluye dummies y variables binarias que creamos antes)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Quito la columna de precio de las variables explicativas para no predecirnos a nosotros mismos
feature_cols_nn = [col for col in numeric_cols if col != "price"]

X_nn = df[feature_cols_nn].copy()

# Parto los datos en entrenamiento y prueba para poder evaluar bien la red
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(
    X_nn,
    y_nn,
    test_size=0.2,
    random_state=100
)

# Convierto a float32 y a matrices de numpy porque así trabaja mejor TensorFlow/Keras
X_train_nn = X_train_nn.astype("float32").values
X_test_nn = X_test_nn.astype("float32").values
y_train_nn = y_train_nn.astype("float32").values
y_test_nn = y_test_nn.astype("float32").values

# Reviso tamaños para confirmar que todo quedó bien armado
print("X_train_nn:", X_train_nn.shape)
print("X_test_nn :", X_test_nn.shape)
print("y_train_nn:", y_train_nn.shape)
print("y_test_nn :", y_test_nn.shape)


X_train_nn: (16661, 98)
X_test_nn : (4166, 98)
y_train_nn: (16661,)
y_test_nn : (4166,)


In [ ]:
import setuptools.dist  # en algunas versiones de Python esto evita problemas al importar tensorflow
import tensorflow as tf

# Fijo semillas para que los resultados sean lo más reproducibles posible
np.random.seed(123)
tf.random.set_seed(123)

# Dimensión de entrada para la red (tantas neuronas como columnas en X)
input_dim = X_train_nn.shape[1]

# Capa de normalización: aprende media y desviación de las variables de entrada
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train_nn)

def build_mlp_model(
    n_hidden_layers=2,
    units_per_layer=64,
    dropout_rate=0.10,
    learning_rate=0.001
):
    """
    Se pasan como argumentos para poder probar varias configuraciones después.
    """
    model = tf.keras.Sequential()
    
    # Entrada + normalización de todas las variables explicativas
    model.add(tf.keras.Input(shape=(input_dim,)))
    model.add(normalizer)
    
    # Capas ocultas densas con ReLU
    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(units_per_layer, activation="relu"))
        if dropout_rate > 0:
            model.add(tf.keras.layers.Dropout(dropout_rate))
    
    # Capa de salida con un solo valor: predicción de log(price)
    model.add(tf.keras.layers.Dense(1))
    
    # Optimizador Adam con tasa de aprendizaje configurable
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    # Uso mean_absolute_error como función de pérdida y métrica principal
    model.compile(
        optimizer=optimizer,
        loss="mean_absolute_error",
        metrics=["mean_absolute_error"]
    )
    
    return model

# Construyo un primer modelo base para revisar que todo esté bien conectado
baseline_model = build_mlp_model(
    n_hidden_layers=2,
    units_per_layer=64,
    dropout_rate=0.10,
    learning_rate=0.001
)

baseline_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 98)             │           197 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,758 (42.03 KB)

 Trainable params: 10,561 (41.25 KB)

 Non-trainable params: 197 (792.00 B)